In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir("..")

In [ ]:
os.getcwd()

# contants

In [ ]:
from customer_support.utils import load_yaml
from dataclasses import dataclass
from dotenv import load_dotenv


load_dotenv()
CONFIG=load_yaml("config/config.yaml")

@dataclass
class RetrievalConstants:
    GEMINI_PRO_NAME=CONFIG.MODELS.REASONER.GEMINI_PRO
    DEEPSEEK_R1_NAME=CONFIG.MODELS.REASONER.DEEPSEEK_R1
    PROMPT_TEMPLATES = {
        "product_bot": """
        You are an expert EcommerceBot specialized in product recommendations and handling customer queries.
        Analyze the provided product titles, ratings, and reviews to provide accurate, helpful responses.
        Stay relevant to the context, and keep your answers concise and informative.

        CONTEXT:
        {context}

        QUESTION: {question}

        YOUR ANSWER:
        """
    }

In [ ]:
print(f"GEMINI_PRO_NAME: {RetrievalConstants.GEMINI_PRO_NAME}")
print(f"DEEPSEEK_R1_NAME: {RetrievalConstants.DEEPSEEK_R1_NAME}")
print("PROMPT_TEMPLATES:", RetrievalConstants.PROMPT_TEMPLATES["product_bot"])

# entity

In [ ]:
from dataclasses import dataclass


@dataclass 
class Retrieval:
    GEMINI_PRO_NAME:str
    DEEPSEEK_R1_NAME:str
    PROMPT_TEMPLATES:dict

# configuration

In [ ]:
from dataclasses import dataclass


@dataclass 
class RetrievalConfig:
    GEMINI_PRO_NAME=RetrievalConstants.GEMINI_PRO_NAME
    DEEPSEEK_R1_NAME=RetrievalConstants.DEEPSEEK_R1_NAME
    PROMPT_TEMPLATES=RetrievalConstants.PROMPT_TEMPLATES

In [ ]:
print(f"GEMINI_PRO_NAME: {RetrievalConfig.GEMINI_PRO_NAME}")
print(f"DEEPSEEK_R1_NAME: {RetrievalConfig.DEEPSEEK_R1_NAME}")
print("PROMPT_TEMPLATES:", RetrievalConfig.PROMPT_TEMPLATES["product_bot"])

# components

In [ ]:
from customer_support.components.data_ingestion import DataIngestionComponents
from customer_support.configuration import DataIngestionConfig
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from customer_support.exception import CustomException 
from langchain_core.prompts import ChatPromptTemplate
from customer_support.logger import logging 
from langchain_groq import ChatGroq
from dataclasses import dataclass 
import sys



@dataclass 
class RetrievalComponents:
    __retrieval_config:Retrieval

    def get_retriever(self) -> None:
        try:
            logging.info("In get_retriever")
            self.data_ingestion_components=DataIngestionComponents(DataIngestionConfig)
            self.retriever=self.data_ingestion_components.get_vector_store().as_retriever()

            logging.info("Out get_retriever")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)
        
    def get_LLM(self):
        """loads large language model either gemini_pro or deepseek_r1
        """
        try:
            logging.info("In get_LLM")
            try:
                self.model_name=self.__retrieval_config.GEMINI_PRO_NAME
                self.llm=ChatGoogleGenerativeAI(model=self.model_name)
                logging.info(f"{self.model_name} loaded")
            except:
                self.model_name=self.__retrieval_config.DEEPSEEK_R1_NAME
                self.llm=ChatGroq(model=self.model_name)
                logging.info(f"{self.model_name} loaded")

            logging.info("Out get_LLM")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)
        
    def get_chain(self):
        """create the chain
        """
        try:
            logging.info("In get_chain")
            self.prompt=ChatPromptTemplate.from_template(self.__retrieval_config.PROMPT_TEMPLATES["product_bot"])
            self.chain=(
                {"context": self.retriever, "question": RunnablePassthrough()}
                | self.prompt
                | self.llm
                | StrOutputParser()
            )
            logging.info("Out get_chain")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)
        
    def invoke(self, query:str) -> str:
        """invokes the chain

        Args:
            query (str): user query

        Returns:
            str: output as string
        """
        try:
            logging.info("In invoke")
            result=self.chain.invoke(query)
            logging.info("Out invoke")
            return result
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys)
        
    def main(self) -> None:
        self.get_retriever()
        self.get_LLM()
        self.get_chain()

# pipeline

In [ ]:
class RetrievalPipeline:
    def __init__(self) -> None:
        self.retrieval=RetrievalComponents(RetrievalConfig)
        self.retrieval.main()
        
    def run(self, query:str) -> str:
        """invokes the pipeline with given query

        Args:
            query (str): query to invoke pipeline

        Returns:
            str: string output of chain
        """
        return self.retrieval.invoke(query)

# main.py

In [ ]:
if __name__ == "__main__":
    retrieval_pipeline = RetrievalPipeline()
    query = "Can you tell me the low budget headphone?"
    result=retrieval_pipeline.run(query)
    print(result)